In [5]:
import cobra
import os
import pandas as pd
from cobra.flux_analysis import flux_variability_analysis


In [6]:
models = {}
model_dir = '../data/modelseed_models/'
for file in os.listdir(model_dir):
    if file.endswith('.json') and not file.endswith('gapfill_removed.json'):
        models[file.split('.json')[0]] = cobra.io.load_json_model(model_dir+file)

In [7]:
# get the mapping of exchange reactions to metabolite names for every exchange reaction
ex_to_name = {}
for species in models.keys():
    ex_rxns = [r.id for r in models[species].reactions if r.id.startswith('EX_')]
    for rxn in ex_rxns:
        if rxn not in ex_to_name.keys():
            ex_to_name[rxn] = models[species].reactions.get_by_id(rxn).reactants[0].name

In [8]:
fva_results = {}
consumed_mets = {}
produced_mets = {}
pfba_solution = {}
pfba_results = {}
pfba_consumed = {}
pfba_produced = {}

for species,model in models.items():
    model.solver='glpk'
    print("performing FVA for {}".format(species))
    ex_rxns = [r.id for r in model.reactions if r.id.startswith('EX_')]
    fva_results[species] = flux_variability_analysis(model,ex_rxns, fraction_of_optimum = 0.05)
    pfba_solution[species] = cobra.flux_analysis.pfba(model)
    
    
#     # get consistently consumed and produced metabolites from exchanges
#     consumed_mets[species] = fva_results[species].loc[(fva_results[species] < -1e-6).sum(axis=1) > 1]
#     produced_mets[species] = fva_results[species].loc[(fva_results[species] > 1e-6).sum(axis=1) > 1]
    
#     pfba_consumed[species] = pfba_solution[species].fluxes.loc[(pfba_solution[species].fluxes < -1e-6)]
#     pfba_produced[species] = pfba_solution[species].fluxes.loc[(pfba_solution[species].fluxes > 1e-6)]


performing FVA for L. lactis
performing FVA for P. copri
performing FVA for B. vulgatus HM-720
performing FVA for E. coli UTI89
performing FVA for L. vaginalis
performing FVA for B. clarus
performing FVA for L. gasseri
performing FVA for C. bolteae
performing FVA for P. distasonis
performing FVA for E. lenta
performing FVA for B. thetaiotaomicron
performing FVA for R. intestinalis
performing FVA for L. sakei subsp. sakei
performing FVA for Akkermansia muciniphila
performing FVA for B. eggerthii
performing FVA for B. animalis subsp. lactis BL-04
performing FVA for B. fragilis HM-714
performing FVA for A. odontolyticus
performing FVA for S. salivarius
performing FVA for C. aerofaciens
performing FVA for B. animalis subsp. lactis BI-07
performing FVA for B. uniformis HM-716
performing FVA for B. dorei
performing FVA for L. salivarius
performing FVA for E. coli HM605
performing FVA for R. bromii
performing FVA for E. eligens
performing FVA for B. fragilis HM-713
performing FVA for A. shahi

In [9]:
# #Now working
# final_pfba = {}
# new_active = {}
# new_active_media = {}
# rxn_list = []

# for species,model in models.items():
#     og_test_mod = models[species].copy()
#     test_mod = models[species].copy()
    
#     with test_mod as model_copy:
#         media_rxns = model_copy.medium.keys()
#         active = pfba_solution[species].fluxes.loc[abs(pfba_solution[species].fluxes) > 0]
#         active_media = active[[rxn for rxn in og_test_mod.medium.keys() if rxn in active.index]]

#         og_active_pfba_df = pd.DataFrame(active_media)
#         og_rxns = [rxn for rxn in og_test_mod.medium.keys() if rxn in active.index]
#         og_medium = model_copy.medium
#         new_medium = og_medium
#         og_optimal = model_copy.optimize().objective_value
#         sorted_by_flux = og_active_pfba_df.sort_values('fluxes',ascending=False)
#         sorted_rxns = sorted_by_flux.axes[0].tolist()        
#         for rxn in sorted_rxns:
#             if rxn not in rxn_list:
#                 if rxn in model_copy.medium:
#                     old_medium = model_copy.medium
#                     new_medium[rxn] = 0
#                     model_copy.medium = new_medium
#                     media_rxns2 = model_copy.medium.keys()
#                     optimal = model_copy.optimize().objective_value #fba
#                     if optimal < 0.05 * og_optimal:
#                         model_copy.medium = old_medium
#                         new_medium = old_medium
#                         rxn_list.append(rxn)
#                     else:
#                         rxn_list.append(rxn)
#                         pfba_iter = cobra.flux_analysis.pfba(model_copy)
#                         active_iter = pfba_iter.fluxes.loc[abs(pfba_iter.fluxes) > 0]
#                         active_media_iter = active[[rxn for rxn in og_test_mod.medium.keys() if rxn in active.index]]
#                         active_pfba_df = pd.DataFrame(active_media)
#                         sorted_by_flux = active_pfba_df.sort_values('fluxes',ascending=False)
#                         sorted_rxns = sorted_by_flux.axes[0].tolist()

#         final_pfba[species] = cobra.flux_analysis.pfba(model_copy)
#         new_active = final_pfba[species].fluxes.loc[abs(final_pfba[species].fluxes) > 0]
#         new_active_media[species] = new_active[[rxn for rxn in og_test_mod.medium.keys() if rxn in new_active.index]]

In [11]:
#Now working
final_pfba = {}
new_active = {}
new_active_media = {}

for species,model in models.items():
    og_test_mod = models[species].copy()
    test_mod = models[species].copy()
    
    with test_mod as model_copy:
        media_rxns = model_copy.medium.keys()
        active = pfba_solution[species].fluxes.loc[abs(pfba_solution[species].fluxes) > 0]
        active_media = active[[rxn for rxn in og_test_mod.medium.keys() if rxn in active.index]]

        og_active_pfba_df = pd.DataFrame(active_media)
        og_rxns = [rxn for rxn in og_test_mod.medium.keys() if rxn in active.index]
        og_medium = model_copy.medium
        new_medium = og_medium
        og_optimal = model_copy.optimize().objective_value
        sorted_by_flux = og_active_pfba_df.sort_values('fluxes',ascending=False)
        sorted_rxns = sorted_by_flux.axes[0].tolist()
        print(sorted_rxns)
        
        while sorted_rxns:
            rxn = sorted_rxns[0]
#             print(rxn)
            if rxn in model_copy.medium:
#                 print(model_copy.medium)
                old_medium = model_copy.medium
                new_medium[rxn] = 0
                model_copy.medium = new_medium
#                 print(model_copy.medium)
                media_rxns2 = model_copy.medium.keys()
                optimal = model_copy.optimize().objective_value #fba
                if optimal < 0.05 * og_optimal:
                    model_copy.medium = old_medium
                    new_medium = old_medium
                    sorted_rxns.remove(rxn)
                else:
                    pfba_iter = cobra.flux_analysis.pfba(model_copy)
                    active_iter = pfba_iter.fluxes.loc[abs(pfba_iter.fluxes) > 0]
                    active_media_iter = active[[rxn for rxn in og_test_mod.medium.keys() if rxn in active.index]]
                    active_pfba_df = pd.DataFrame(active_media)
                    sorted_by_flux = active_pfba_df.sort_values('fluxes',ascending=False)
                    sorted_rxns = sorted_by_flux.axes[0].tolist()

        final_pfba[species] = cobra.flux_analysis.pfba(model_copy)
        new_active = final_pfba[species].fluxes.loc[abs(final_pfba[species].fluxes) > 0]
        new_active_media[species] = new_active[[rxn for rxn in og_test_mod.medium.keys() if rxn in new_active.index]]

['EX_cpd00100_e', 'EX_cpd00001_e', 'EX_cpd00314_e', 'EX_cpd00222_e', 'EX_cpd00027_e', 'EX_cpd00122_e', 'EX_cpd00011_e', 'EX_cpd00064_e', 'EX_cpd00007_e', 'EX_cpd00154_e', 'EX_cpd00030_e', 'EX_cpd00205_e', 'EX_cpd00254_e', 'EX_cpd00264_e', 'EX_cpd00099_e', 'EX_cpd00048_e', 'EX_cpd00034_e', 'EX_cpd00149_e', 'EX_cpd00063_e', 'EX_cpd03696_e', 'EX_cpd00058_e', 'EX_cpd00305_e', 'EX_cpd10516_e', 'EX_cpd00220_e', 'EX_cpd00028_e', 'EX_cpd00010_e', 'EX_cpd00355_e', 'EX_cpd00092_e', 'EX_cpd00084_e', 'EX_cpd00309_e', 'EX_cpd11584_e', 'EX_cpd01080_e', 'EX_cpd00226_e', 'EX_cpd00069_e', 'EX_cpd00060_e', 'EX_cpd00066_e', 'EX_cpd00051_e', 'EX_cpd00039_e', 'EX_cpd00322_e', 'EX_cpd11583_e', 'EX_cpd00080_e', 'EX_cpd00013_e', 'EX_cpd00158_e', 'EX_cpd00794_e', 'EX_cpd00276_e', 'EX_cpd00067_e']
EX_cpd00100_e
{'EX_cpd00067_e': 1000.0, 'EX_cpd00075_e': 1000.0, 'EX_cpd11576_e': 1000.0, 'EX_cpd04097_e': 1000.0, 'EX_cpd01030_e': 1000.0, 'EX_cpd01012_e': 1000.0, 'EX_cpd00276_e': 1000.0, 'EX_cpd00082_e': 1000.0, 'E

KeyboardInterrupt: 

In [ ]:
new_df = pd.DataFrame(new_active_media)
new_df

In [ ]:
new_df = new_df.fillna(0).T
new_df

In [ ]:
new_df = new_df.divide(abs(new_df).max(axis=1),axis=0)
exclude = ['EX_cpd00001_e','EX_cpd00007_e'] # H2O, Oxygen, 
new_df = new_df[[col for col in new_df.columns if col not in exclude]]
new_df = new_df[new_df.columns[abs(new_df).max() > 0.05]]
new_df = new_df[new_df.columns[new_df.min() < -0.00001]]
# new_df = new_df[new_df.columns[new_df.max() > -0.9]]


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as colors
# set the colormap and centre the colorbar - from http://chris35wills.github.io/matplotlib_diverging_colorbar/
class MidpointNormalize(colors.Normalize):
	"""
	Normalise the colorbar so that diverging bars work there way either side from a prescribed midpoint value)

	e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-100, vmax=100))
	"""
	def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
		self.midpoint = midpoint
		colors.Normalize.__init__(self, vmin, vmax, clip)

	def __call__(self, value, clip=None):
		# I'm ignoring masked values and all kinds of edge cases to make a
		# simple example...
		x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
		return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))


In [ ]:
fig,ax = plt.subplots()
plt.rcParams["font.family"] = "Arial"



elev_min=new_df.min().min()
elev_max=new_df.max().max()
mid_val=0


im= ax.imshow(new_df.values, cmap='bwr', clim=(elev_min, elev_max), norm=MidpointNormalize(midpoint=mid_val,vmin=elev_min, vmax=elev_max))

# construct the custom colormap

#im = ax.imshow(con_prod_frame.values, cmap = 'bwr')
ax.set_xticks(np.arange(len(new_df.columns)))
ax.set_yticks(np.arange(len(new_df.index)))
ax.set_ylim((-0.5,len(new_df.index)-0.5))
labels = [ex_to_name[exchange] for exchange in new_df.columns]
ax.set_xticklabels(labels, rotation = 45, ha = 'right', size=16)
# convert xtick labels to metabolite names

ax.set_yticklabels(new_df.index, size=16)
plt.colorbar(im)
fig.set_size_inches(60,30)
plt.savefig('../results/pfba_threshold5.svg')
new_df.to_csv

In [16]:
model_copy.medium = new_medium
model_copy.medium

{'EX_cpd00067_e': 1000.0,
 'EX_cpd00075_e': 1000.0,
 'EX_cpd11576_e': 1000.0,
 'EX_cpd04097_e': 1000.0,
 'EX_cpd01030_e': 1000.0,
 'EX_cpd01012_e': 1000.0,
 'EX_cpd00276_e': 1000.0,
 'EX_cpd00082_e': 1000.0,
 'EX_cpd00001_e': 1000.0,
 'EX_cpd00058_e': 1000.0,
 'EX_cpd00080_e': 1000.0,
 'EX_cpd00314_e': 1000.0,
 'EX_cpd00222_e': 1000.0,
 'EX_cpd00013_e': 1000.0,
 'EX_cpd00051_e': 1000.0,
 'EX_cpd00064_e': 1000.0,
 'EX_cpd00179_e': 1000.0,
 'EX_cpd00355_e': 1000.0,
 'EX_cpd00060_e': 1000.0,
 'EX_cpd00076_e': 1000.0,
 'EX_cpd01171_e': 1000.0,
 'EX_cpd00023_e': 1000.0,
 'EX_cpd00281_e': 1000.0,
 'EX_cpd00154_e': 1000.0,
 'EX_cpd03724_e': 1000.0,
 'EX_cpd00637_e': 1000.0,
 'EX_cpd00122_e': 1000.0,
 'EX_cpd08306_e': 1000.0,
 'EX_cpd03696_e': 1000.0,
 'EX_cpd00309_e': 1000.0,
 'EX_cpd00220_e': 1000.0,
 'EX_cpd00084_e': 1000.0,
 'EX_cpd00588_e': 1000.0,
 'EX_cpd00254_e': 1000.0,
 'EX_cpd00138_e': 1000.0,
 'EX_cpd00039_e': 1000.0,
 'EX_cpd00540_e': 1000.0,
 'EX_cpd10516_e': 1000.0,
 'EX_cpd0097